# Criação de dataset

A função desse Jupyter é ler os arquivos txts e montar um Dataframe.

Os sinais foram exportados do Matlab e cada um dos arquivos txt é referente a um canal de um dos sinais (até o momento temos apenas um sianl).

* Importando as bibliotecas

In [1]:
import glob
import logging
import numpy as np
import pandas as pd

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
arquivos = glob.glob("sinais/*.txt")
arquivos

['sinais/SF_10_S1_canal_1.txt',
 'sinais/SF_10_S1_canal_10.txt',
 'sinais/SF_10_S1_canal_11.txt',
 'sinais/SF_10_S1_canal_12.txt',
 'sinais/SF_10_S1_canal_13.txt',
 'sinais/SF_10_S1_canal_14.txt',
 'sinais/SF_10_S1_canal_15.txt',
 'sinais/SF_10_S1_canal_2.txt',
 'sinais/SF_10_S1_canal_3.txt',
 'sinais/SF_10_S1_canal_4.txt',
 'sinais/SF_10_S1_canal_5.txt',
 'sinais/SF_10_S1_canal_6.txt',
 'sinais/SF_10_S1_canal_7.txt',
 'sinais/SF_10_S1_canal_8.txt',
 'sinais/SF_10_S1_canal_9.txt',
 'sinais/SF_10_S2_canal_1.txt',
 'sinais/SF_10_S2_canal_10.txt',
 'sinais/SF_10_S2_canal_11.txt',
 'sinais/SF_10_S2_canal_12.txt',
 'sinais/SF_10_S2_canal_13.txt',
 'sinais/SF_10_S2_canal_14.txt',
 'sinais/SF_10_S2_canal_15.txt',
 'sinais/SF_10_S2_canal_2.txt',
 'sinais/SF_10_S2_canal_3.txt',
 'sinais/SF_10_S2_canal_4.txt',
 'sinais/SF_10_S2_canal_5.txt',
 'sinais/SF_10_S2_canal_6.txt',
 'sinais/SF_10_S2_canal_7.txt',
 'sinais/SF_10_S2_canal_8.txt',
 'sinais/SF_10_S2_canal_9.txt',
 'sinais/SF_10_S3_canal_1.tx

In [4]:
arquivos[0]

'sinais/SF_10_S1_canal_1.txt'

In [5]:
teste = arquivos[0].split("_canal")[0].split("_S")[1]
teste

'1'

In [6]:
teste[0].split("_S")

['1']

* Lendo todos os sinais e criando o Dataframe

In [15]:
%%time

sinal = []

dataframes = []

for sinal in arquivos:
    logging.info(f'Lendo o sinal: {sinal}')
    lines = list(open(sinal, 'r'))
    
    logging.info(f'Removendo espaços e \t')
    lines = [line.replace("\t", " ") for line in lines]
    lines = [line.rstrip() for line in lines]
    
    logging.info(f'Transformando em lista')
    lines = lines[0].split(" ")

    logging.info('Separando a amostra em 2000')
    sinais = np.array_split(lines, 2000)
    
    logging.info('Passando para dataframe')
    df = pd.DataFrame(sinais).T
    df.dropna(inplace=True)
    df["label"] = 2
    df.iloc[:15]["label"] = 0
    df.iloc[:15]["label"] = 0
    df.iloc[-15:]["label"] = 1
    df = df[df["label"] != 2]

    df["semana"] = sinal.split("_canal")[0].split("_S")[1]
    
    dataframes.append(df)
    
    logging.info('---------------------------------------')

INFO:root:Lendo o sinal: sinais/SF_10_S1_canal_1.txt
INFO:root:Removendo espaços e 	
INFO:root:Transformando em lista
INFO:root:Separando a amostra em 2000
INFO:root:Passando para dataframe
<timed exec>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

CPU times: user 2min 9s, sys: 5.75 s, total: 2min 15s
Wall time: 2min 19s


Concatenando dos os dataframes em um único Dataset

In [16]:
dados = pd.concat(dataframes)

In [17]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 91
Columns: 2002 entries, 0 to semana
dtypes: int64(1), object(2001)
memory usage: 275.1+ MB


Reiniciando o index do Dataset

In [18]:
dados.reset_index(inplace=True)

In [19]:
dados.tail()

,index,0,1,2,3,4,5,6,7,8,...,1992,1993,1994,1995,1996,1997,1998,1999,label,semana
17995,87,3,6,4,4,3,3,4,4,2,...,4,3,4,3,3,4,3,3,1,4
17996,88,2,4,4,3,3,4,3,4,3,...,4,3,3,2,3,3,4,3,1,4
17997,89,2,3,6,3,2,3,3,3,4,...,4,3,4,2,3,5,3,3,1,4
17998,90,3,1,4,3,1,2,2,4,5,...,3,4,4,3,3,3,3,3,1,4
17999,91,2,2,4,5,3,3,2,4,5,...,3,4,4,3,4,4,4,4,1,4


Dropando a coluna Index que foi criada

In [20]:
dados.drop(['index'], inplace=True, axis=1)

In [21]:
dados.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,label,semana
13445,-3,-2,-2,-2,-4,-3,-1,-2,-3,-3,...,-3,-4,-4,-2,-2,-2,-3,-4,0,2
3435,-25,4,18,-44,17,-97,87,419,192,-206,...,-186,499,-158,102,6,-64,-172,-33,1,4
6589,-8,74,243,49,-142,-144,-12,-64,13,32,...,221,-71,326,-371,85,-79,-18,-185,1,3
13337,46,2,-15,-39,-47,92,-30,-66,-30,119,...,-11,74,-36,-85,-26,-93,72,-211,1,2
789,-4,-21,-29,1,-5,-16,-15,-45,-15,-8,...,-7,0,-91,-10,-9,-44,11,-100,0,2


In [22]:
len(dados)

18000

```python
dados['feat'] = dados['feat'].fillna(value=dados['feat'].median())
```

In [23]:
dados.to_csv('dados_FEM_10_voluntarios_c_semanas_15_sinais_noout.csv', index=False)